# 모듈, 데이터 불러오기

In [1]:
!pip install folium matplotlib mapclassify

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 67.9 MB/s eta 0:00:00


In [3]:
import pandas as pd
import geopandas as gpd

In [4]:
df_grid = pd.read_csv('/content/drive/MyDrive/프로젝트1/데이터/unique_grid.csv')
df_grid.head(2)

,gid,lbl,val,geometry,gu
0,다사58a48a,6085.0,6085.0,"POLYGON ((958000 1948000, 958000 1948500, 9585...",강남구
1,다사58b48a,3446.0,3446.0,"POLYGON ((958500 1948000, 958500 1948500, 9590...",강남구


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
traffic_light = pd.read_csv('/content/drive/MyDrive/프로젝트1/데이터/도로시설물/서울시 신호등 관련 정보.csv', encoding = 'cp949')
traffic_light.head(2)

,부착대관리번호,상태 (공통),부착방식,부착대길이,고가 (공통),부착대방향,신호등수량,배면등수량,신호등종류,배면등종류,...,작업구분 (공통),표출구분 (공통),신규정규화ID,공사관리번호,부착대관리번호.1,이력ID,위치정보,X좌표,Y좌표,공사형태 (공통)
0,03-0000024658,1.0,2.0,NaN,1.0,348,1.0,0.0,7.0,NaN,...,1,2.0,NaN,2006-0101-047,03-024658,61177,NaN,206327.81700,561650.62207,NaN
1,03-0000049718,1.0,2.0,999.0,1.0,259,1.0,0.0,7.0,NaN,...,2,1.0,NaN,2000-0000-000,03-049718,57699,NaN,192663.57583,554312.49703,NaN


# 데이터 정제
- 컬럼중에서 부착대관리번호, X좌표, Y좌표 컬럼만 추출

In [7]:
traffic_light.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113606 entries, 0 to 113605
Data columns (total 25 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   부착대관리번호    113606 non-null  object 
 1   상태 (공통)    113104 non-null  float64
 2   부착방식       113146 non-null  float64
 3   부착대길이      106968 non-null  float64
 4   고가 (공통)    103970 non-null  float64
 5   부착대방향      113606 non-null  int64  
 6   신호등수량      113604 non-null  float64
 7   배면등수량      113586 non-null  float64
 8   신호등종류      113572 non-null  float64
 9   배면등종류      32582 non-null   float64
 10  설치일        11815 non-null   float64
 11  교체일        11727 non-null   float64
 12  지주관리번호     113606 non-null  object 
 13  신호발광구분     113606 non-null  int64  
 14  제조회사       18842 non-null   object 
 15  작업구분 (공통)  113606 non-null  int64  
 16  표출구분 (공통)  113600 non-null  float64
 17  신규정규화ID    12265 non-null   float64
 18  공사관리번호     109827 non-null  object 
 19  부착대관리번호.1  113606 non-n

In [8]:
traffic_light = traffic_light.rename(columns={'부착대관리번호':'신호등관리번호'})

In [9]:
traffic_light = traffic_light.loc[ :, ['신호등관리번호', 'X좌표' , 'Y좌표']]
traffic_light.info()
traffic_light.head(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113606 entries, 0 to 113605
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   신호등관리번호  113606 non-null  object 
 1   X좌표      113606 non-null  float64
 2   Y좌표      113606 non-null  float64
dtypes: float64(2), object(1)
memory usage: 2.6+ MB


,신호등관리번호,X좌표,Y좌표
0,03-0000024658,206327.817,561650.62207


# pandas ⟶ geopandas

## 격자지도 데이터 프레임
```
기존에 .shp을 로드해서 csv로 저장했기 때문에 'geometry'열이 존재한다.

그러나 csv로 저장하면서 기존에 'geometry'열의 데이터 타입이 문자열로 변경되었다.

따라서 wkt.loads를 이용해서 문자열 타입을 폴리곤 타입으로 바꾸고, geodataframe을 구성할 때 좌표계 정보(epsg:5179)를 주었다.

```

In [10]:
from shapely import wkt

df_grid['geometry'] = df_grid['geometry'].apply(wkt.loads)
geo_grids = gpd.GeoDataFrame(df_grid, crs='epsg:5179')

# 신호등 데이터 프레임
```
기존에 .csv 파일이었기 때문에 'geometry'열이 존재하지 않는다.

따라서 geodataframe을 구성할 때 x좌표와 y좌표를 이용하여 'geometry' 컬럼을 만들어준다. 

그 후 to_crs() 함수를 이용해 좌표계를 5179 좌표계로 변경한다.

좌표계 설정 이유 : 좌표계의 종류가 많기 때문에 어떤 종류의 좌표계인지 명시해주어야 함

```

In [11]:
geo_traffic_light = gpd.GeoDataFrame(
    traffic_light, geometry=gpd.points_from_xy(traffic_light['X좌표'], traffic_light['Y좌표'])
)
geo_traffic_light.set_crs(epsg = 5186, inplace = True)
geo_traffic_light = geo_traffic_light.to_crs(epsg=5179)

In [17]:
geo_traffic_light.explore(tiles='http://mt0.google.com/vt/lyrs=m&hl=ko&x={x}&y={y}&z={z}', attr='Google')

## geopandas 의 sjoin을 사용

- 정확히는 모르지만 공간에 따라 데이터를 join 하는 것 같음
- pandas.join과 유사 ( 다른 점: pandas의 join은 데이터가 같은 것끼리 엮지만 geopandas의 sjoin은 공간이 겹치면 엮는다 )

In [ ]:
join_geo_df = gpd.sjoin(geo_traffic_light, geo_grids, how='left')
join_geo_df.shape

In [ ]:
# 피벗테이블 이용해서 격자지도별로 신호등 카운트
traffic_light_by_grid = join_geo_df.pivot_table(index='gid', values='신호등관리번호', aggfunc='count')
traffic_light_by_grid.head(5)

In [ ]:
traffic_light_by_grid.to_csv('traffic_light_by_grid')